# Model Comparison: YOLO vs Detectron2

In this notebook, we will compare the performance of different computer vision models, including YOLOv5, YOLOv8, and various Detectron2 architectures. We will evaluate their accuracy, speed, and other relevant metrics on a common dataset.

## Models to Compare:
- **YOLOv5**: Fast single-stage detector
- **YOLOv8**: Latest YOLO with improved architecture  
- **Faster R-CNN**: Two-stage detector from Detectron2
- **RetinaNet**: Single-stage detector with focal loss
- **Mask R-CNN**: Instance segmentation capable detector

In [ ]:
# Import necessary libraries
from models.detectron2_model import Detectron2Model
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import cv2

# Add src to path
sys.path.append('../src')

# from models.yolo_v5 import YOLOv5  # Uncomment when available
# from models.yolo_v8 import YOLOv8  # Uncomment when available
# from data.dataset_loader import load_dataset  # Uncomment when available
# from utils.metrics import calculate_metrics  # Uncomment when available

# Load the dataset (placeholder for now)
# dataset = load_dataset('path/to/dataset')

# Initialize models (placeholder for YOLO models)
# yolo_v5_model = YOLOv5(weights='path/to/yolov5/weights')
# yolo_v8_model = YOLOv8(weights='path/to/yolov8/weights')

# Evaluate models (placeholder)
# yolo_v5_results = yolo_v5_model.evaluate(dataset)
# yolo_v8_results = yolo_v8_model.evaluate(dataset)

# Calculate metrics (placeholder)
# yolo_v5_metrics = calculate_metrics(yolo_v5_results)
# yolo_v8_metrics = calculate_metrics(yolo_v8_results)

# Compare metrics (placeholder)
# metrics_comparison = pd.DataFrame({
#     'YOLOv5': yolo_v5_metrics,
#     'YOLOv8': yolo_v8_metrics
# })

# Plot comparison (placeholder)
# metrics_comparison.plot(kind='bar')
# plt.title('Model Comparison')
# plt.ylabel('Metrics')
# plt.xlabel('Metric Type')
# plt.show()

print("Imports configured for Detectron2 model comparison")
print("YOLO imports are commented out - uncomment when models are available")

In [ ]:
# Initialize all models for comparison
print("Initializing models...")

# YOLO models (uncomment when weights are available)
# yolo_v5_model = YOLOv5(weights='path/to/yolov5/weights')
# yolo_v8_model = YOLOv8(weights='path/to/yolov8/weights')

# Detectron2 models
try:
    detectron2_models = {
        'Faster R-CNN': Detectron2Model('faster_rcnn', num_classes=80),
        'RetinaNet': Detectron2Model('retinanet', num_classes=80),
        'Mask R-CNN': Detectron2Model('mask_rcnn', num_classes=80),
        'FCOS': Detectron2Model('fcos', num_classes=80)
    }
    print("Model initialization completed!")

    # Model information
    for name, model in detectron2_models.items():
        info = model.get_model_info()
        print(f"{name}: {info['architecture']} detector")

except Exception as e:
    print(f"Error initializing Detectron2 models: {e}")
    print("Make sure Detectron2 is properly installed")

In [ ]:
# Performance benchmarking function
def benchmark_model(model, test_images, model_name="Unknown", num_runs=10):
    """Benchmark model performance on test images"""
    results = {
        'model_name': model_name,
        'inference_times': [],
        'fps': 0,
        'total_detections': 0,
        'avg_confidence': 0
    }

    print(f"Benchmarking {model_name}...")

    # Warm up runs
    for _ in range(3):
        _ = model.predict(test_images[0])

    # Actual benchmark
    all_detections = []
    start_time = time.time()

    for i in range(min(num_runs, len(test_images))):
        img_start = time.time()
        predictions = model.predict(test_images[i])
        img_end = time.time()

        results['inference_times'].append(img_end - img_start)

        # Extract detection info (Detectron2 format)
        if hasattr(predictions, '__getitem__') and 'instances' in predictions:
            num_dets = len(predictions['instances'])
            if num_dets > 0:
                confidences = predictions['instances'].scores.cpu().numpy()
                results['total_detections'] += num_dets
                all_detections.extend(confidences)

    end_time = time.time()

    # Calculate metrics
    avg_inference_time = np.mean(results['inference_times'])
    results['fps'] = 1.0 / avg_inference_time
    results['avg_confidence'] = np.mean(
        all_detections) if all_detections else 0
    results['std_inference_time'] = np.std(results['inference_times'])

    print(
        f"  Average inference time: {avg_inference_time:.3f}s ± {results['std_inference_time']:.3f}s")
    print(f"  FPS: {results['fps']:.1f}")
    print(f"  Total detections: {results['total_detections']}")
    print(f"  Average confidence: {results['avg_confidence']:.3f}")

    return results

In [ ]:
# Prepare test data
# For demonstration, create sample images (replace with your actual test dataset)
def create_sample_images(num_images=5, height=480, width=640):
    """Create sample test images"""
    images = []
    for i in range(num_images):
        # Create random image with some simple shapes for testing
        img = np.random.randint(0, 255, (height, width, 3), dtype=np.uint8)

        # Add some simple geometric shapes to make detection more interesting
        cv2.circle(img, (width//4, height//4), 50, (255, 0, 0), -1)
        cv2.rectangle(img, (width//2, height//2),
                      (width//2 + 100, height//2 + 80), (0, 255, 0), -1)

        images.append(img)
    return images


# Create test images (replace with actual test dataset loading)
test_images = create_sample_images(10)
print(f"Created {len(test_images)} test images for benchmarking")

# Or load actual test dataset (uncomment when available)
# test_images = load_dataset('path/to/test/dataset')

In [ ]:
# Run benchmarks on all models
benchmark_results = {}

# Benchmark Detectron2 models
for name, model in detectron2_models.items():
    results = benchmark_model(model, test_images, name,
                              num_runs=len(test_images))
    benchmark_results[name] = results

# Benchmark YOLO models (uncomment when models are available)
# yolo_v5_results = benchmark_model(yolo_v5_model, test_images, "YOLOv5")
# yolo_v8_results = benchmark_model(yolo_v8_model, test_images, "YOLOv8")
# benchmark_results["YOLOv5"] = yolo_v5_results
# benchmark_results["YOLOv8"] = yolo_v8_results

print("\\nBenchmarking completed!")

In [ ]:
# Visualize comparison results
def plot_comparison_results(results):
    \"\"\"Create comprehensive comparison plots\"\"\"
    model_names = list(results.keys())
    fps_values = [results[name]['fps'] for name in model_names]
    detection_counts = [results[name]['total_detections'] for name in model_names]
    avg_confidences = [results[name]['avg_confidence'] for name in model_names]
    
    # Create subplots
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
    
    # Colors for bars
    colors = ['skyblue', 'lightcoral', 'lightgreen', 'gold', 'orange', 'purple'][:len(model_names)]
    
    # 1. FPS Comparison
    bars1 = ax1.bar(model_names, fps_values, color=colors)
    ax1.set_ylabel('Frames Per Second (FPS)')
    ax1.set_title('Inference Speed Comparison')
    ax1.tick_params(axis='x', rotation=45)
    
    # Add value labels
    for bar, fps in zip(bars1, fps_values):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
                 f'{fps:.1f}', ha='center', va='bottom')
    
    # 2. Detection Count Comparison
    bars2 = ax2.bar(model_names, detection_counts, color=colors)
    ax2.set_ylabel('Total Detections')
    ax2.set_title('Detection Count Comparison')
    ax2.tick_params(axis='x', rotation=45)
    
    for bar, count in zip(bars2, detection_counts):
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
                 f'{count}', ha='center', va='bottom')
    
    # 3. Average Confidence Comparison
    bars3 = ax3.bar(model_names, avg_confidences, color=colors)
    ax3.set_ylabel('Average Confidence Score')
    ax3.set_title('Detection Confidence Comparison')
    ax3.tick_params(axis='x', rotation=45)
    ax3.set_ylim(0, 1)
    
    for bar, conf in zip(bars3, avg_confidences):
        ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
                 f'{conf:.3f}', ha='center', va='bottom')
    
    # 4. Speed vs Accuracy Scatter Plot
    ax4.scatter(fps_values, avg_confidences, c=colors[:len(model_names)], s=100, alpha=0.7)
    ax4.set_xlabel('FPS (Speed)')
    ax4.set_ylabel('Average Confidence (Accuracy Proxy)')
    ax4.set_title('Speed vs Accuracy Trade-off')
    
    # Add model name labels
    for i, name in enumerate(model_names):
        ax4.annotate(name, (fps_values[i], avg_confidences[i]), 
                    xytext=(5, 5), textcoords='offset points', fontsize=9)
    
    plt.tight_layout()
    plt.show()

# Plot the results
plot_comparison_results(benchmark_results)

## Conclusion

In this notebook, we compared the performance of YOLOv5 and YOLOv8 on a common dataset. The results indicate that [insert findings here]. Further analysis can be conducted to explore the strengths and weaknesses of each model.